In [4]:
# pip install zat pandas

In [25]:
import gzip
from zat.log_to_dataframe import LogToDataFrame
import pandas as pd
import tempfile

# File path to the conn.log.gz
file_path = "../zeek/logs/2024-12-16/dhcp.10:07:28-11:00:00.log.gz"

# Create a temporary uncompressed log file
with gzip.open(file_path, 'rb') as gzipped_file:
    # Decompress the file content
    with tempfile.NamedTemporaryFile(mode='wb', delete=False) as temp_file:
        temp_file.write(gzipped_file.read())
        uncompressed_path = temp_file.name


log_to_df = LogToDataFrame()
df = log_to_df.create_dataframe(uncompressed_path)
df.info()
df.head()



<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6080 entries, 2024-12-16 10:07:17.393410921 to 2024-12-16 10:59:40.196435928
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype          
---  ------          --------------  -----          
 0   uids            6080 non-null   object         
 1   client_addr     2 non-null      category       
 2   server_addr     2 non-null      category       
 3   mac             6080 non-null   category       
 4   host_name       6077 non-null   category       
 5   client_fqdn     0 non-null      category       
 6   domain          0 non-null      category       
 7   requested_addr  6 non-null      category       
 8   assigned_addr   2 non-null      category       
 9   lease_time      2 non-null      timedelta64[ns]
 10  client_message  0 non-null      category       
 11  server_message  0 non-null      category       
 12  msg_types       6080 non-null   category       
 13  duration        6080 non-null   timed

,uids,client_addr,server_addr,mac,host_name,client_fqdn,domain,requested_addr,assigned_addr,lease_time,client_message,server_message,msg_types,duration
ts,,,,,,,,,,,,,,
2024-12-16 10:07:17.393410921,COb3ME3PSqM0nj4uzj,NaN,NaN,aa:b3:b9:a4:de:d1,MikroTik,NaN,NaN,NaN,NaN,NaT,NaN,NaN,DISCOVER,0 days
2024-12-16 10:07:17.393423080,COb3ME3PSqM0nj4uzj,NaN,NaN,78:9a:18:73:92:f0,MikroTik,NaN,NaN,NaN,NaN,NaT,NaN,NaN,DISCOVER,0 days
2024-12-16 10:07:16.368320942,COb3ME3PSqM0nj4uzj,NaN,NaN,78:9a:18:73:92:f0,MikroTik,NaN,NaN,NaN,NaN,NaT,NaN,NaN,DISCOVER,0 days
2024-12-16 10:07:18.314089060,COb3ME3PSqM0nj4uzj,NaN,NaN,aa:b3:b9:a4:de:d1,MikroTik,NaN,NaN,NaN,NaN,NaT,NaN,NaN,DISCOVER,0 days
2024-12-16 10:07:13.296777010,COb3ME3PSqM0nj4uzj,NaN,NaN,aa:b3:b9:a4:de:d1,MikroTik,NaN,NaN,NaN,NaN,NaT,NaN,NaN,DISCOVER,0 days



### Key Observations in Your Output
1. **DataFrame Structure**:
   - Index: `DatetimeIndex` with precise timestamps.
   - Total Columns: 20 columns with appropriate data types.
   - Memory-efficient types: 
     - `category` for strings (e.g., `id.orig_h`, `proto`, `service`).
     - `timedelta64` for `duration` column.
     - `UInt16` and `UInt64` for numeric data.

2. **Non-Null Counts**:
   - Some columns (e.g., `service`, `duration`) have missing values.
   - The `tunnel_parents` column has all `NaN` values.

3. **Performance**:
   - Optimized memory usage due to appropriate data types.

---

### What You Can Do Next:
Here are a few ideas to explore further:

1. **Filter the DataFrame**:
   Extract records based on specific conditions, such as filtering `dns` or `quic` services.
   ```python
   dns_records = df[df['service'] == 'dns']
   print(dns_records.head())
   ```

2. **Convert Timestamps to Seconds**:
   If you need duration in seconds for calculations:
   ```python
   df['duration_sec'] = df['duration'].dt.total_seconds()
   print(df[['duration', 'duration_sec']].head())
   ```

3. **Analyze Missing Data**:
   Identify and handle missing values in columns like `service` and `duration`:
   ```python
   missing_summary = df.isnull().sum()
   print("Missing Data Summary:\n", missing_summary)

   # Fill missing service with 'unknown'
   df['service'] = df['service'].fillna('unknown')
   ```

4. **Group and Aggregate**:
   Analyze connections grouped by protocols, for example:
   ```python
   proto_summary = df.groupby('proto')['uid'].count().reset_index()
   print(proto_summary)
   ```

5. **Save to Parquet for Future Analysis**:
   Save the cleaned DataFrame as a Parquet file for efficient storage and retrieval:
   ```python
   df.to_parquet("conn_cleaned.parquet", engine="pyarrow")
   print("DataFrame saved to conn_cleaned.parquet")
   ```

---

### Example: Quick Summary Stats
Here’s a quick summary to explore the DataFrame's contents:
```python
print("Summary Statistics:")
print(df.describe(include='all'))
```

This will give you insights into:
- Count, mean, and range of numerical columns (like `orig_bytes`, `resp_bytes`).
- Unique values for categorical columns.

In [20]:
df[df['service'] == 'dns'].head()

,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents
ts,,,,,,,,,,,,,,,,,,,,
2024-12-16 09:59:54.975208998,CgflZ13ybYBXTEgAvi,192.168.127.114,56266,192.168.127.134,53,udp,dns,0 days 00:00:00.055739,50,176,SF,T,T,0,Dd,1,78,1,204,NaN
2024-12-16 09:59:54.975614071,CyU9mpNNl606E5uJi,192.168.127.114,34226,192.168.127.134,53,udp,dns,0 days 00:00:00.061160,50,234,SF,T,T,0,Dd,1,78,1,262,NaN
2024-12-16 09:59:55.037101984,C9yxrt2diu1QPx2Xwa,192.168.127.114,50785,192.168.127.134,53,udp,dns,0 days 00:00:00.047208,60,137,SF,T,T,0,Dd,1,88,1,165,NaN
2024-12-16 10:00:00.200301886,CuUJzJ4Juyy6xIhHh,192.168.127.114,54767,192.168.127.134,53,udp,dns,0 days 00:00:00.056300,46,101,SF,T,T,0,Dd,1,74,1,129,NaN
2024-12-16 10:00:00.261168002,C948dq1BkMtjMaxDE5,192.168.127.114,38190,192.168.127.134,53,udp,dns,0 days 00:00:00.007068,46,46,SF,T,T,0,Dd,1,74,1,74,NaN


In [21]:
df['duration_sec'] = df['duration'].dt.total_seconds()
df.head()

,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,...,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,duration_sec
ts,,,,,,,,,,,,,,,,,,,,,
2024-12-16 09:59:54.975208998,CgflZ13ybYBXTEgAvi,192.168.127.114,56266,192.168.127.134,53,udp,dns,0 days 00:00:00.055739,50,176,...,T,T,0,Dd,1,78,1,204,NaN,0.055739
2024-12-16 09:59:54.975614071,CyU9mpNNl606E5uJi,192.168.127.114,34226,192.168.127.134,53,udp,dns,0 days 00:00:00.061160,50,234,...,T,T,0,Dd,1,78,1,262,NaN,0.061160
2024-12-16 09:59:55.037101984,C9yxrt2diu1QPx2Xwa,192.168.127.114,50785,192.168.127.134,53,udp,dns,0 days 00:00:00.047208,60,137,...,T,T,0,Dd,1,88,1,165,NaN,0.047208
2024-12-16 09:59:05.721227884,CX98VA2t5shazpJxr,192.168.127.114,55102,172.217.25.202,443,udp,quic,0 days 00:00:00.806031,7340,6784,...,T,F,0,Dd,13,7704,16,7232,NaN,0.806031
2024-12-16 10:00:00.200301886,CuUJzJ4Juyy6xIhHh,192.168.127.114,54767,192.168.127.134,53,udp,dns,0 days 00:00:00.056300,46,101,...,T,T,0,Dd,1,74,1,129,NaN,0.056300


In [22]:
missing_summary = df.isnull().sum()
print("Missing Data Summary:\n", missing_summary)

Missing Data Summary:
 uid                  0
id.orig_h            0
id.orig_p            0
id.resp_h            0
id.resp_p            0
proto                0
service            840
duration           798
orig_bytes         798
resp_bytes         798
conn_state           0
local_orig           0
local_resp           0
missed_bytes         0
history             11
orig_pkts            0
orig_ip_bytes        0
resp_pkts            0
resp_ip_bytes        0
tunnel_parents    2796
duration_sec       798
dtype: int64


In [23]:
proto_summary = df.groupby('proto')['uid'].count().reset_index()
print(proto_summary)

  proto   uid
0  icmp    11
1   tcp   162
2   udp  2623


/tmp/ipykernel_102222/3775064666.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  proto_summary = df.groupby('proto')['uid'].count().reset_index()


In [24]:
df.describe(include='all')

,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,...,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,duration_sec
count,2796,2796,2796.0,2796,2796.0,2796,1956,1998,1998.0,1998.0,...,2796,2796,2796.0,2785,2796.0,2796.0,2796.0,2796.0,0,1998.000000
unique,2796,61,<NA>,107,<NA>,3,7,NaN,<NA>,<NA>,...,2,2,<NA>,68,<NA>,<NA>,<NA>,<NA>,0,NaN
top,CkSpaiywbgMHriaoj,192.168.28.229,<NA>,192.168.28.1,<NA>,udp,dns,NaN,<NA>,<NA>,...,T,T,<NA>,D,<NA>,<NA>,<NA>,<NA>,NaN,NaN
freq,1,1195,<NA>,645,<NA>,2623,1404,NaN,<NA>,<NA>,...,2795,1507,<NA>,1447,<NA>,<NA>,<NA>,<NA>,NaN,NaN
mean,NaN,NaN,36683.160229,NaN,5948.72568,NaN,NaN,0 days 00:00:14.910760481,2466.097097,2646.765766,...,NaN,NaN,533.629471,NaN,6.354435,2019.110157,5.554006,1553.973534,NaN,14.910760
std,NaN,NaN,20129.092164,NaN,9156.04949,NaN,NaN,0 days 00:01:34.353317881,11202.349981,13366.765585,...,NaN,NaN,9439.609432,NaN,23.911972,10275.634882,24.912475,5969.612305,NaN,94.353318
min,NaN,NaN,3.0,NaN,0.0,NaN,NaN,0 days 00:00:00.000001,0.0,0.0,...,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.000001
25%,NaN,NaN,5678.0,NaN,53.0,NaN,NaN,0 days 00:00:00.007784500,40.0,0.0,...,NaN,NaN,0.0,NaN,1.0,66.0,0.0,0.0,NaN,0.007784
50%,NaN,NaN,42628.5,NaN,5353.0,NaN,NaN,0 days 00:00:00.141793,210.0,84.0,...,NaN,NaN,0.0,NaN,1.0,118.0,0.0,0.0,NaN,0.141793
75%,NaN,NaN,52944.75,NaN,8610.0,NaN,NaN,0 days 00:00:03.993344250,3330.5,4466.25,...,NaN,NaN,0.0,NaN,4.0,812.0,1.0,232.0,NaN,3.993344
